In [5]:
import torch
print(torch.cuda.is_available())

False


In [3]:
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd')
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

Using cache found in /Users/anita/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
ssd_model.to('cuda')
ssd_model.eval()

In [ ]:
uris = ['Car Model.v4i.coco/test']

In [ ]:
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs)

In [ ]:
with torch.no_grad(): 
    detections_batch = ssd_model(tensor)

In [ ]:
results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0,40 ) for results in results_per_input]

In [ ]:
classes_to_labels = utils.get_coco_object_dictionary()

In [ ]:
from matplotlib import pyplot as plt 
import matplotlib.patches as patches

for image_idx in range(len(best_results_per_input)): 
    fig, ax = plt.subplots(1)
    
    image = inputs[image_idx]/2 + 0.5 
    ax.imgshow(image)
    
    bbboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bbboxes)): 
        left, bot, right, top = bbboxes[idx]
        x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
plt.show()